<a href="https://colab.research.google.com/github/venkatharshithyadav/EY-Data-Challenge/blob/main/EY_Datascience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv("UHI Experiment Sample Benchmark Notebook V7.csv")  # Update with actual data file
satellite_features = pd.read_csv("Sentinel2_Features.csv")  # Sentinel-2 extracted features
weather_data = pd.read_csv("WeatherData.csv")  # Weather information

# Merge datasets
df = df.merge(satellite_features, on='location_id', how='left')
df = df.merge(weather_data, on='location_id', how='left')

# Feature Engineering
df['NDVI'] = (df['B8'] - df['B4']) / (df['B8'] + df['B4'])  # Normalized Difference Vegetation Index
df['NDBI'] = (df['B11'] - df['B8']) / (df['B11'] + df['B8'])  # Normalized Difference Built-up Index
df['Temperature_Humidity_Index'] = df['Temperature'] / (df['Humidity'] + 1e-3)  # Avoid division by zero

# Select Features
features = ['NDVI', 'NDBI', 'Temperature', 'Humidity', 'WindSpeed', 'SolarFlux', 'BuildingDensity']
X = df[features]
y = df['UHI_Index']

# Standardize Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# XGBoost Regressor (Replaces RandomForest)
xgb_model = xgb.XGBRegressor(n_estimators=500, max_depth=6, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8)
xgb_model.fit(X_train, y_train)

# Predict and Evaluate
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
final_score = (r2_train + r2_test) / 2  # Average R² Score

print(f"Train R²: {r2_train:.4f}, Test R²: {r2_test:.4f}, Final Score: {final_score:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: 'UHI_data.csv'

In [ ]:
pip install rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 68.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load training data
training_data_path = "/content/sample_data/Training_data_uhi_index_2025-02-18.csv"
data = pd.read_csv(training_data_path)

# Preprocess data (dropping longitude and latitude as required)
data = data.drop(columns=['Longitude', 'Latitude'])

# Handle datetime feature
data['datetime'] = pd.to_datetime(data['datetime'])
data['hour'] = data['datetime'].dt.hour
data['month'] = data['datetime'].dt.month
data = data.drop(columns=['datetime'])

# Split dataset
X = data.drop(columns=['UHI Index'])
y = data['UHI Index']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.0002629006104769664


<ipython-input-4-478dbdeb10f9>:14: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['datetime'] = pd.to_datetime(data['datetime'])


In [6]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define training data path
training_data_path = "/content/sample_data/Training_data_uhi_index_2025-02-18.csv"

# Check if file exists
if not os.path.exists(training_data_path):
    raise FileNotFoundError(f"Training data file not found at {training_data_path}. Please check the path.")

# Load training data
data = pd.read_csv(training_data_path)

# Ensure required columns exist
required_columns = {'Longitude', 'Latitude', 'datetime', 'UHI Index'}
missing_columns = required_columns - set(data.columns)
if missing_columns:
    raise ValueError(f"Missing columns in dataset: {missing_columns}")

# Preprocess data (dropping longitude and latitude as required)
data = data.drop(columns=['Longitude', 'Latitude'])

# Handle datetime feature
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
if data['datetime'].isnull().any():
    raise ValueError("Invalid datetime values found in dataset.")

data['hour'] = data['datetime'].dt.hour
data['month'] = data['datetime'].dt.month
data = data.drop(columns=['datetime'])

# Split dataset
X = data.drop(columns=['UHI Index'])
y = data['UHI Index']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.0002629006104769664


<ipython-input-6-365de8a0fe5d>:27: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')


<ipython-input-8-7797cbc48938>:31: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')


Mean Squared Error: 0.0002629006104769664


KeyError: 'datetime'

In [11]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define training data path
training_data_path = "/content/sample_data/Training_data_uhi_index_2025-02-18.csv"
validation_data_path = "/content/sample_data/Submission_template_UHI2025-v2.csv"
prediction_output_path = "/mnt/data/UHI_predictions.csv"

# Check if file exists
if not os.path.exists(training_data_path):
    raise FileNotFoundError(f"Training data file not found at {training_data_path}. Please check the path.")
if not os.path.exists(validation_data_path):
    raise FileNotFoundError(f"Validation data file not found at {validation_data_path}. Please check the path.")

# Load training data
data = pd.read_csv(training_data_path)

# Ensure required columns exist
required_columns = {'Longitude', 'Latitude', 'UHI Index'}
missing_columns = required_columns - set(data.columns)
if missing_columns:
    raise ValueError(f"Missing columns in dataset: {missing_columns}")

# Keep only necessary features
data = data[['Longitude', 'Latitude', 'UHI Index']]

# Split dataset
X = data.drop(columns=['UHI Index'])
y = data['UHI Index']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Load validation dataset
validation_data = pd.read_csv(validation_data_path)

# Ensure validation data has the same features as training
validation_data = validation_data[['Longitude', 'Latitude']]

# Predict on validation set
validation_data['Predicted UHI Index'] = model.predict(validation_data)

# Save predictions to CSV
submission = pd.DataFrame({'ID': range(len(validation_data)), 'Predicted UHI Index': validation_data['Predicted UHI Index']})
submission.to_csv(prediction_output_path, index=False)
print(f"Predictions saved to {prediction_output_path}")


Mean Squared Error: 8.018202165035353e-06


OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [14]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define training data path
training_data_path = "/content/sample_data/Training_data_uhi_index_2025-02-18.csv"
validation_data_path = "/content/sample_data/Submission_template_UHI2025-v2.csv"
prediction_output_path = "/mnt/data/UHI_predictions.csv"

# Check if file exists
if not os.path.exists(training_data_path):
    raise FileNotFoundError(f"Training data file not found at {training_data_path}. Please check the path.")
if not os.path.exists(validation_data_path):
    raise FileNotFoundError(f"Validation data file not found at {validation_data_path}. Please check the path.")

# Load training data
data = pd.read_csv(training_data_path)

# Ensure required columns exist
required_columns = {'Longitude', 'Latitude', 'UHI Index'}
missing_columns = required_columns - set(data.columns)
if missing_columns:
    raise ValueError(f"Missing columns in dataset: {missing_columns}")

# Keep only necessary features
data = data[['Longitude', 'Latitude', 'UHI Index']]

# Split dataset
X = data.drop(columns=['UHI Index'])
y = data['UHI Index']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Load validation dataset
validation_data = pd.read_csv(validation_data_path)

# Ensure validation data has the same features as training
validation_data = validation_data[['Longitude', 'Latitude']]

# Predict on validation set
validation_data['Predicted UHI Index'] = model.predict(validation_data)

# Ensure output directory exists
output_dir = os.path.dirname(prediction_output_path)
os.makedirs(output_dir, exist_ok=True)

# Save predictions to CSV with correct columns
submission = validation_data[['Longitude', 'Latitude']].copy()
submission['UHI Index'] = validation_data['Predicted UHI Index']  # Update UHI Index with predicted values

# Save file
submission.to_csv("/content/UHI_predictions.csv", index=False)
print("Predictions saved to /content/UHI_predictions.csv")


Mean Squared Error: 8.018202165035353e-06
Predictions saved to /content/UHI_predictions.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [19]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Define training data path
training_data_path = "/content/sample_data/Training_data_uhi_index_2025-02-18.csv"
validation_data_path = "/content/sample_data/Submission_template_UHI2025-v2.csv"
prediction_output_path = "/mnt/data/UHI_predictions.csv"

# Load training data
data = pd.read_csv(training_data_path)

# Ensure required columns exist
required_columns = {'Longitude', 'Latitude', 'UHI Index'}
missing_columns = required_columns - set(data.columns)
if missing_columns:
    raise ValueError(f"Missing columns in dataset: {missing_columns}")

# Feature Engineering: Add squared terms & interactions
data['Longitude²'] = data['Longitude'] ** 2
data['Latitude²'] = data['Latitude'] ** 2
data['Longitude * Latitude'] = data['Longitude'] * data['Latitude']

# Split dataset
X = data.drop(columns=['UHI Index'])
y = data['UHI Index']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_model = None
best_score = float("inf")

for train_idx, test_idx in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Model Stacking
    rf = RandomForestRegressor(n_estimators=300, max_depth=20, min_samples_split=5, random_state=42)
    gb = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=5, random_state=42)
    xgb_model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=5, random_state=42)

    rf.fit(X_train, y_train)
    gb.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)

    # Predict and average models
    y_pred_rf = rf.predict(X_test)
    y_pred_gb = gb.predict(X_test)
    y_pred_xgb = xgb_model.predict(X_test)

    y_pred_ensemble = (y_pred_rf + y_pred_gb + y_pred_xgb) / 3  # Ensemble average

    mse = mean_squared_error(y_test, y_pred_ensemble)
    print(f"Fold MSE: {mse}")

    if mse < best_score:
        best_score = mse
        best_model = rf  # Choose the best performing model

print(f"Best Model MSE: {best_score}")

# Load validation dataset
validation_data = pd.read_csv(validation_data_path)

# Feature Engineering for Validation Set
validation_data['Longitude²'] = validation_data['Longitude'] ** 2
validation_data['Latitude²'] = validation_data['Latitude'] ** 2
validation_data['Longitude * Latitude'] = validation_data['Longitude'] * validation_data['Latitude']

# Scale validation data
validation_scaled = scaler.transform(validation_data[['Longitude', 'Latitude', 'Longitude²', 'Latitude²', 'Longitude * Latitude']])

# Predict on validation set using best model
validation_data['UHI Index'] = best_model.predict(validation_scaled)

# Save predictions to CSV in correct format
output_dir = os.path.dirname(prediction_output_path)
os.makedirs(output_dir, exist_ok=True)

validation_data[['Longitude', 'Latitude', 'UHI Index']].to_csv(prediction_output_path, index=False)
print(f"Predictions saved to {prediction_output_path}")


ValueError: could not convert string to float: '24-07-2021 15:53'

In [23]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Define data paths
training_data_path = "/content/sample_data/Training_data_uhi_index_2025-02-18.csv"
validation_data_path = "/content/sample_data/Submission_template_UHI2025-v2.csv"
prediction_output_path = "/content/sample_data/UHI_predictions.csv"

# Load training data
data = pd.read_csv(training_data_path)

# Check if 'datetime' column exists and convert it
if 'datetime' in data.columns:
    data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce', dayfirst=True)
    data['hour'] = data['datetime'].dt.hour
    data['month'] = data['datetime'].dt.month
    data['weekday'] = data['datetime'].dt.weekday
    data.drop(columns=['datetime'], inplace=True)

# Ensure necessary columns exist
required_columns = {'Longitude', 'Latitude', 'UHI Index'}
missing_columns = required_columns - set(data.columns)
if missing_columns:
    raise ValueError(f"Missing columns in dataset: {missing_columns}")

# Feature Engineering: Add squared terms & interactions
data['Longitude²'] = data['Longitude'] ** 2
data['Latitude²'] = data['Latitude'] ** 2
data['Longitude * Latitude'] = data['Longitude'] * data['Latitude']

# Select features
features = ['Longitude', 'Latitude', 'Longitude²', 'Latitude²', 'Longitude * Latitude', 'hour', 'month', 'weekday']
data = data.dropna(subset=features)  # Ensure no NaN values before training

# Split dataset
X = data[features]
y = data['UHI Index']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train model with K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_model = None
best_score = float("inf")

for train_idx, test_idx in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    if mse < best_score:
        best_score = mse
        best_model = model

print(f"Best Model MSE: {best_score}")

# Load validation dataset
validation_data = pd.read_csv(validation_data_path)

# Convert datetime in validation set if present
if 'datetime' in validation_data.columns:
    validation_data['datetime'] = pd.to_datetime(validation_data['datetime'], errors='coerce', dayfirst=True)
    validation_data['hour'] = validation_data['datetime'].dt.hour
    validation_data['month'] = validation_data['datetime'].dt.month
    validation_data['weekday'] = validation_data['datetime'].dt.weekday
    validation_data.drop(columns=['datetime'], inplace=True)
else:
    # If datetime is missing, assign default values (e.g., mean or most common)
    validation_data['hour'] = 12  # Default hour
    validation_data['month'] = 6  # Default month (June)
    validation_data['weekday'] = 3  # Default weekday (Wednesday)

# Feature Engineering on validation set
validation_data['Longitude²'] = validation_data['Longitude'] ** 2
validation_data['Latitude²'] = validation_data['Latitude'] ** 2
validation_data['Longitude * Latitude'] = validation_data['Longitude'] * validation_data['Latitude']

# Ensure validation data has the same features
validation_data = validation_data[features]

# Scale validation data
validation_scaled = scaler.transform(validation_data)

# Predict on validation set
validation_data['UHI Index'] = best_model.predict(validation_scaled)

# Ensure output directory exists
output_dir = os.path.dirname(prediction_output_path)
os.makedirs(output_dir, exist_ok=True)

# Save predictions to CSV
validation_data[['Longitude', 'Latitude', 'UHI Index']].to_csv(prediction_output_path, index=False)
print(f"Predictions saved to {prediction_output_path}")


Best Model MSE: 8.098844552746147e-06
Predictions saved to /content/sample_data/UHI_predictions.csv


In [25]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Define paths
training_data_path = "/content/sample_data/Training_data_uhi_index_2025-02-18.csv"
validation_data_path = "/content/sample_data/Submission_template_UHI2025-v2.csv"
prediction_output_path = "/content/sample_data/UHI_predictions.csv"

# Load training data
data = pd.read_csv(training_data_path)

# Ensure required columns exist
required_columns = {'Longitude', 'Latitude', 'UHI Index'}
missing_columns = required_columns - set(data.columns)
if missing_columns:
    raise ValueError(f"Missing columns in dataset: {missing_columns}")

# Feature Engineering: Add interaction terms & polynomial features
data['Longitude²'] = data['Longitude'] ** 2
data['Latitude²'] = data['Latitude'] ** 2
data['Longitude * Latitude'] = data['Longitude'] * data['Latitude']

# Add higher-order polynomial features
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly_features = poly.fit_transform(data[['Longitude', 'Latitude']])
poly_feature_names = poly.get_feature_names_out(['Longitude', 'Latitude'])

# Create dataframe for polynomial features and merge
poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)
data = pd.concat([data, poly_df], axis=1)

# Prepare dataset
features = poly_feature_names
X = data[features]
y = data['UHI Index']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train Model using K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_model = None
best_score = float("inf")

# Define model with hyperparameter tuning
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

random_search = RandomizedSearchCV(
    xgb_model, param_distributions=param_dist,
    n_iter=20, cv=3, scoring='neg_mean_squared_error',
    random_state=42, n_jobs=-1
)
random_search.fit(X_scaled, y)
best_model = random_search.best_estimator_

# Final evaluation
y_pred = best_model.predict(X_scaled)
mse = mean_squared_error(y, y_pred)
print(f"Best Model MSE: {mse}")

# Load validation dataset
validation_data = pd.read_csv(validation_data_path)

# Feature Engineering on validation set
validation_data['Longitude²'] = validation_data['Longitude'] ** 2
validation_data['Latitude²'] = validation_data['Latitude'] ** 2
validation_data['Longitude * Latitude'] = validation_data['Longitude'] * validation_data['Latitude']

# Generate polynomial features for validation set
validation_poly_features = poly.transform(validation_data[['Longitude', 'Latitude']])
validation_poly_df = pd.DataFrame(validation_poly_features, columns=poly_feature_names)
validation_data = pd.concat([validation_data, validation_poly_df], axis=1)

# Ensure validation data has the same features
validation_data = validation_data[features]

# Scale validation data
validation_scaled = scaler.transform(validation_data)

# Predict on validation set
validation_data['UHI Index'] = best_model.predict(validation_scaled)

# Ensure output directory exists
output_dir = os.path.dirname(prediction_output_path)
os.makedirs(output_dir, exist_ok=True)

# Save predictions to CSV
validation_data[['Longitude', 'Latitude', 'UHI Index']].to_csv(prediction_output_path, index=False)
print(f"Predictions saved to {prediction_output_path}")


Best Model MSE: 0.00011587550663688153
Predictions saved to /content/sample_data/UHI_predictions.csv
